# Preprocessing Pipeline



In [114]:
import os
from os.path import join as opj
from nipype.interfaces.freesurfer import ReconAll, Surface2VolTransform, MRIsCalc, ApplyVolTransform
from nipype.interfaces.fsl import BET ,FAST
from nipype.pipeline.engine import Workflow, Node, MapNode, JoinNode
from nipype.interfaces.utility import IdentityInterface
from nipype.workflows.smri.freesurfer.utils import getdefaultconfig
import nipype.interfaces.io as nio 
from nipype import DataGrabber, SelectFiles, DataSink

from nipype.workflows.smri.freesurfer import create_reconall_workflow

## Defining Directories

In [115]:
subj_dir = '/home/adway/Desktop/Symbiosis/FCD_rawdata/Symbiosis/'
fs_folder = '/home/adway/freesurfer'
datasink_dir = 'datasink'
exp_dir  = os.path.abspath('nipypeexpdir')
out_dir = os.path.abspath('/home/adway/Desktop/Symbiosis/nipypeout')
if not os.path.isdir(exp_dir) | os.path.isdir(out_dir):
    os.mkdir(exp_dir)
    os.mkdir(out_dir)
    
# List of Subjects that we want to run the workflow on, append to this list

subjlist = ['s1001','s1002']
            #,'s1003','s1004','s1006','s1007','s1008','s1009','s1010','s1011','s1014','s1015',
            #'s1017','s1018','s1019','s1020','s1021','s1022','s1023','s1024','s1025','s1026','s1027','s1028','s1029','s1030']


In [116]:
import nipype.interfaces.freesurfer as fs
fs.FSCommand.set_default_subjects_dir('/home/adway/freesurfer')
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")


## Create main workflow

In [117]:
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subjlist)]

In [142]:
# BET - Skullstipping

#bet = MapNode(BET(frac = 0.5, robust = True, output_type='NIFTI_GZ'),iterfield = ['in_file'],name = "Bet")

# Recon-all (includes skull stripping)
subjects_dir = os.path.join(subj_dir , 'subject_dir')
reconall = MapNode(ReconAll(directive = 'all',subjects_dir = subj_dir), iterfield = ['subject_id', 'T1_files'], name= "reconall")


# Local Gyrification Index Calculation

localgi = MapNode(ReconAll(directive = 'localGI', subjects_dir = subj_dir),
                   iterfield = ['subject_id'],name = "LocalGI")

#Interhemispheric Asymmetry

interasymm = MapNode(Surface2VolTransform(subjects_dir = subj_dir, surf_name = 'white'), 
                     iterfield = ['source_file', 'hemi'], name = "Surf2voltrans",)

#Intermediate Calculations
intermcalc1  = MapNode(MRIsCalc(action = 'sub', out_file = 'lh.nii.gz'), 
                   name = "Mriscalc_1", iterfield = ['in_file1','in_file2'])
intermcalc2  = MapNode(MRIsCalc(action = 'sub', out_file = 'rh.nii.gz'), 
                   name = "Mriscalc_2", iterfield = ['in_file1','in_file2'])
intermcalc3  = Node(MRIsCalc(action = 'add', out_file = 'interasymm.nii.gz'), 
                   name = "Mriscalc_3")

convert2vol = MapNode(ApplyVolTransform(fs_target = True),iterfield = ['source_file','subject'], name = 'Vol2Voltrans')

In [143]:
T1_file = opj('{subject_id}','{subject_id}_t1_mprage_*.nii.gz')
T2_file = opj('{subject_id}','{subject_id}_t2.nii.gz')


templates = {'T1' : T1_file ,
             'T2' : T2_file}

fileselect = MapNode(SelectFiles(templates), iterfield = ['subject_id'],name='fileselect')

fileselect.inputs.base_directory = '/home/adway/Desktop/Symbiosis/FCD_rawdata/Symbiosis/'
fileselect.inputs.subject_id = subjlist

# Datasink

datasink = Node(DataSink(base_directory = exp_dir), name= 'Datasink')



In [144]:
reconwf = Workflow(name = "reconallwf")
reconwf.base_dir = os.path.join(exp_dir , 'reconallwfdir')
reconwf.connect([(infosource ,fileselect , [('subject_id', 'subject_id')]),
                 (fileselect, reconall , [('T1', 'T1_files')]),
                 (infosource, reconall , [('subject_id', 'subject_id')])
                ])
                 

In [136]:
reconwf.run("MultiProc", plugin_args={'n_procs': 4})

190728-08:58:17,926 nipype.workflow INFO:
	 Workflow reconallwf settings: ['check', 'execution', 'logging', 'monitoring']
190728-08:58:17,973 nipype.workflow INFO:
	 Running in parallel.
190728-08:58:17,987 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 13.98/13.98, Free processors: 4/4.
190728-08:58:19,616 nipype.workflow INFO:
	 [Node] Setting-up "reconallwf.fileselect" in "/home/adway/Desktop/nipypeexpdir/reconallwfdir/reconallwf/_subject_id_s1002/fileselect".
190728-08:58:19,618 nipype.workflow INFO:
	 [Node] Setting-up "reconallwf.fileselect" in "/home/adway/Desktop/nipypeexpdir/reconallwfdir/reconallwf/_subject_id_s1001/fileselect".
190728-08:58:19,636 nipype.workflow INFO:
	 [Node] Setting-up "_fileselect0" in "/home/adway/Desktop/nipypeexpdir/reconallwfdir/reconallwf/_subject_id_s1002/fileselect/mapflow/_fileselect0".
190728-08:58:19,639 nipype.workflow INFO:
	 [Node] Setting-up "_fileselect0" in "/home/adway/Desktop/nipypeexpdir/recona

190729-10:23:18,691 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 0 jobs ready. Free memory (GB): 13.78/13.98, Free processors: 3/4.
                     Currently running:
                       * reconallwf.reconall
190729-10:37:31,226 nipype.workflow INFO:
	 [Node] Finished "_reconall0".
190729-10:37:31,264 nipype.workflow INFO:
	 [Node] Finished "reconallwf.reconall".
190729-10:37:33,206 nipype.workflow INFO:
	 [Job 4] Completed (reconallwf.reconall).
190729-10:37:33,212 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 13.98/13.98, Free processors: 4/4.
190729-10:37:33,979 nipype.workflow INFO:
	 [Node] Setting-up "reconallwf.LocalGI" in "/home/adway/Desktop/nipypeexpdir/reconallwfdir/reconallwf/_subject_id_s1001/LocalGI".
190729-10:37:33,985 nipype.workflow INFO:
	 [Node] Setting-up "_LocalGI0" in "/home/adway/Desktop/nipypeexpdir/reconallwfdir/reconallwf/_subject_id_s1001/LocalGI/mapflow/_LocalGI0".
190729-10:37:33,994 nipype.wor

RuntimeError: Workflow did not execute cleanly. Check log for details

In [ ]:
#!nipypecli crash -r $(pwd)/crash-20190729-103737-adway-LocalGI.a0-8ae8692c-e6f0-47a9-bec2-4fe3883c62b6.pklz

In [141]:
localgiwf = Workflow(name = 'localgiwf')
localgiwf.base_dir = os.path.join(exp_dir, 'localgiwf')
localgiwf.connect([(reconwf, localgi, [('reconall.subject_id', 'subject_id')])
                  ])

localgiwf.run("MultiProc", plugin_args = {'n_procs' : 4})